# Test FANET Logistic Regression
Date: 25/05/2023

## Imports

In [1]:
import tensorflow as tf
import pandas as pd # for data manipulation 
import numpy as np
import glob, math
from scipy import special
from sklearn.metrics import accuracy_score

def euclidean_dist(row):
    # Function to calc euclidean distance on every df row 
    euc_dist = math.sqrt(row["U2G_Distance"]**2 - row["Height"]**2)
    return euc_dist

def q_func(x):
    q = 0.5 - 0.5*special.erf(x / np.sqrt(2))
    return q

def friis_calc(P,freq,dist,ple):
    '''
    Friis path loss equation
    P = Tx transmit power
    freq = Signal frequency
    dist = Transmission distance
    ple = Path loss exponent
    '''
    propagation_speed = 299792458
    l = propagation_speed / freq
    h_pl = P * l**2 / (16*math.pi**2)
    P_Rx = h_pl * dist**(-ple)
    return P_Rx

def plos_calc(h_dist, height_tx, height_rx, env='suburban'):
    '''
    % This function implements the LoS probability model from the paper
    % "Blockage Modeling for Inter-layer UAVs Communications in Urban
    % Environments" 
    % param h_dist    : horizontal distance between Tx and Rx (m)
    % param height_tx : height of Tx
    % param height_rx : height of Rx
    '''
    if env == 'suburban':
        a1 = 0.1
        a2 = 7.5e-4
        a3 = 8
    
    delta_h = height_tx - height_rx
    # pow_factor = 2 * h_dist * math.sqrt(a1*a2/math.pi) + a1 # NOTE: Use this pow_factor if assuming PPP building dist.
    pow_factor = h_dist * math.sqrt(a1*a2) # NOTE: Use this pow_factor if assuming ITU-R assumptions.
    if delta_h == 0:
        p = (1 - math.exp((-(height_tx)**2) / (2*a3**2))) ** pow_factor
    else:
        if delta_h < 0:
            h1 = height_rx
            h2 = height_tx
        else:
            h1 = height_tx
            h2 = height_rx
        delta_h = abs(delta_h)
        p = (1 - (math.sqrt(2*math.pi)*a3 / delta_h) * abs(q_func(h1/a3) - q_func(h2/a3))) ** pow_factor
    return p

def sinr_lognormal_approx(h_dist, height, env='suburban'):
    '''
    To approximate the SNR from signal considering multipath fading and shadowing
    Assuming no interference due to CSMA, and fixed noise
    Inputs:
    h_dist = Horizontal Distance between Tx and Rx
    height = Height difference between Tx and Rx
    env = The operating environment (currently only suburban supported)
    '''
    # Signal properties
    P_Tx_dBm = 20 # Transmit power of 
    P_Tx = 10**(P_Tx_dBm/10) / 1000
    freq = 2.4e9 # Channel frequency (Hz)
    noise_dBm = -86
    noise = 10**(noise_dBm/10) / 1000
    if env == "suburban":
        # ENV Parameters Constants ----------------------------------
        # n_min = 2
        # n_max = 2.75
        # K_dB_min = 7.8
        # K_dB_max = 17.5
        # K_min = 10**(K_dB_min/10)
        # K_max = 10**(K_dB_max/10)
        # alpha = 11.25 # Env parameters for logarithm std dev of shadowing 
        # beta = 0.06 # Env parameters for logarithm std dev of shadowing 
        n_min = 2
        n_max = 2.75
        K_dB_min = 1.4922
        K_dB_max = 12.2272
        K_min = 10**(K_dB_min/10)
        K_max = 10**(K_dB_max/10)
        alpha = 11.1852 # Env parameters for logarithm std dev of shadowing 
        beta = 0.06 # Env parameters for logarithm std dev of shadowing 
        # -----------------------------------------------------------
    # Calculate fading parameters
    PLoS = plos_calc(h_dist, 0, height, env='suburban')
    theta_Rx = math.atan2(height, h_dist) * 180 / math.pi # Elevation angle in degrees
    ple = (n_min - n_max) * PLoS + n_max # Path loss exponent
    sigma_phi_dB = alpha*math.exp(-beta*theta_Rx)
    sigma_phi = 10**(sigma_phi_dB/10) # Logarithmic std dev of shadowing
    K = K_min * math.exp(math.log(K_max/K_min) * PLoS**2)
    omega = 1 # Omega of NCS (Rician)
    dist = math.sqrt(h_dist**2 + height**2)
    P_Rx = friis_calc(P_Tx, freq, dist, ple)
    # Approximate L-NCS RV (which is the SNR) as lognormal
    eta = math.log(10) / 10
    mu_phi = 10*math.log10(P_Rx)
    E_phi = math.exp(eta*mu_phi + eta**2*sigma_phi**2/2) # Mean of shadowing RV
    var_phi = math.exp(2*eta*mu_phi+eta**2*sigma_phi**2)*(math.exp(eta**2*sigma_phi**2)-1) # Variance of shadowing RV
    E_chi = (special.gamma(1+1)/(1+K))*special.hyp1f1(-1,1,-K)*omega
    var_chi = (special.gamma(1+2)/(1+K)**2)*special.hyp1f1(-2,1,-K)*omega**2 - E_chi**2
    E_SNR = E_phi * E_chi / noise # Theoretical mean of SINR
    var_SNR = ((var_phi+E_phi**2)*(var_chi+E_chi**2) - E_phi**2 * E_chi**2) / noise**2
    std_dev_SNR = math.sqrt(var_SNR)
    # sigma_ln = math.sqrt(math.log(var_SNR/E_SNR**2 + 1))
    # mu_ln = math.log(E_SNR) - sigma_ln**2/2
    return E_SNR, std_dev_SNR

2023-07-03 11:20:10.202658: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 11:20:10.317926: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-03 11:20:10.322719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-03 11:20:10.322732: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Load Logistic Regression Models for each Outputs

In [4]:
# Uplink
# reliability_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_ul/reliability_model.005-0.1774.h5", compile=False)
# incr_rcvd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_ul/incr_rcvd_model.005-0.0366.h5", compile=False)
# queue_overflow_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_ul/queue_overflow_model.005-0.2665.h5", compile=False)
# delay_excd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_ul/delay_excd_model.005-0.1733.h5", compile=False)
# Downlink
# reliability_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_dl/reliability_model.005-0.1544.h5", compile=False)
# incr_rcvd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_dl/incr_rcvd_model.005-0.1224.h5", compile=False)
# queue_overflow_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_dl/queue_overflow_model.005-0.3246.h5", compile=False)
# delay_excd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_dl/delay_excd_model.005-0.1853.h5", compile=False)
# Video
reliability_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_vid/reliability_model.005-0.1355.h5", compile=False)
incr_rcvd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_vid/incr_rcvd_model.005-0.1800.h5", compile=False)
queue_overflow_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_vid/queue_overflow_model.005-0.3771.h5", compile=False)
delay_excd_model = tf.keras.models.load_model("/home/research-student/omnet-fanet/nn_checkpoints/lr_multimodulation_video_nosinr_vid/delay_excd_model.005-0.1157.h5", compile=False)

# Compile models
reliability_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
incr_rcvd_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
queue_overflow_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
delay_excd_model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

## Test using Taguchi Test Dataset

In [14]:
# Set minimum probability for failure mode to be considered
MIN_FAILURE_PROB = 0.01

# Load test dataset
test_data_df = pd.read_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_MultiModulation_Hovering_NoVideo/Multi_Modulation_Test_Cases_2_Downlink.csv")

# Calculate mean and std dev of SINR
test_data_df[['Mean_SINR',"Std_Dev_SINR"]] = test_data_df.apply(lambda row: sinr_lognormal_approx(row['Horizontal_Distance'],row['Height']),axis=1,result_type='expand')
test_data_df["Mean_SINR"] = test_data_df["Mean_SINR"].apply(lambda x: 10*math.log10(x))
test_data_df["Std_Dev_SINR"] = test_data_df["Std_Dev_SINR"].apply(lambda x: 10*math.log10(x))

mean_sinr = test_data_df["Mean_SINR"].values
std_dev_sinr = test_data_df["Std_Dev_SINR"].values

# Normalize inputs
max_mean_sinr = 10*math.log10(1123) # The max mean SINR calculated at (0,60) is 1122.743643457063 (linear)
max_std_dev_sinr = 10*math.log10(466) # The max std dev SINR calculated at (0,60) is 465.2159856885714 (linear)
min_mean_sinr = 10*math.log10(0.2) # The min mean SINR calculated at (1200,60) is 0.2251212887895188 (linear)
min_std_dev_sinr = 10*math.log10(0.7) # The min std dev SINR calculated at (1200,300) is 0.7160093126585219 (linear)

norm_mean_sinr = [2*(m-min_mean_sinr) / (max_mean_sinr-min_mean_sinr) - 1 for m in mean_sinr]
norm_std_dev_sinr = [2*(s-min_std_dev_sinr) / (max_std_dev_sinr-min_std_dev_sinr) - 1 for s in std_dev_sinr]
norm_uav_send_int = test_data_df["UAV_Sending_Interval"].replace({10:-1, 20:-0.5, 40:0, 100:0.5, 1000:1}).values
norm_modulation = test_data_df["Modulation"].replace({"BPSK":1, "QPSK":0.3333, "QAM16":-0.3333, "QAM64":-1}).values

# For storing prediction results
predicted_reliability = []
predicted_incr_rcvd = [] 
predicted_delay_excd = []
predicted_queue_overflow = []

# Run inference
model_inputs = list(zip(norm_mean_sinr, norm_std_dev_sinr, norm_uav_send_int, norm_modulation))
predicted_reliability = reliability_model.predict(model_inputs)
predicted_incr_rcvd = incr_rcvd_model.predict(model_inputs)
predicted_delay_excd = delay_excd_model.predict(model_inputs)
predicted_queue_overflow = queue_overflow_model.predict(model_inputs)
# print(prediction)

# Save the results to CSV
test_data_df['Predicted_Reliability'] = predicted_reliability
test_data_df['Predicted_Queue_Overflow_Prob'] = predicted_queue_overflow
test_data_df['Predicted_Incr_Rcvd_Prob'] = predicted_incr_rcvd
test_data_df['Predicted_Delay_Excd_Prob'] = predicted_delay_excd

test_data_df["Reliability_Class"] = pd.cut(test_data_df["Reliability"], bins=[-0.1,0.2,0.5,0.8,1], labels=["Low", "ModeratelyLow", "ModeratelyHigh", "High"])
test_data_df["Predicted_Reliability_Class"] = pd.cut(test_data_df["Predicted_Reliability"], bins=[-0.1,0.2,0.5,0.8,1], labels=["Low", "ModeratelyLow", "ModeratelyHigh", "High"])
test_data_df["Failure_Mode"] = test_data_df[["Queue_Overflow_Prob", "Incorrectly_Rcvd_Prob", "Delay_Excd_Prob"]].idxmax(axis=1)
test_data_df["Predicted_Failure_Mode"] = test_data_df[["Predicted_Queue_Overflow_Prob", "Predicted_Incr_Rcvd_Prob", "Predicted_Delay_Excd_Prob"]].idxmax(axis=1)
# Replace label for Failure Mode with "None" if none of the failure modes have a probability > 5%
test_data_df.loc[(test_data_df["Queue_Overflow_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Incorrectly_Rcvd_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Delay_Excd_Prob"] < MIN_FAILURE_PROB),["Failure_Mode"]] = "None"
test_data_df.loc[(test_data_df["Predicted_Queue_Overflow_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Predicted_Incr_Rcvd_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Predicted_Delay_Excd_Prob"] < MIN_FAILURE_PROB),["Predicted_Failure_Mode"]] = "None"

# Compute the model accuracy and mean abs err
failure_mode = test_data_df["Failure_Mode"].replace({"Queue_Overflow_Prob":1, "Incorrectly_Rcvd_Prob":2, "Delay_Excd_Prob":3, "None":4})
failure_mode_predicted = test_data_df["Predicted_Failure_Mode"].replace({"Predicted_Queue_Overflow_Prob":1, "Predicted_Incr_Rcvd_Prob":2, "Predicted_Delay_Excd_Prob":3, "None":4})
reliability_accuracy = accuracy_score(test_data_df["Reliability_Class"], test_data_df["Predicted_Reliability_Class"])
failure_mode_accuracy = accuracy_score(failure_mode, failure_mode_predicted)
reliability_mae = np.mean(abs(test_data_df['Reliability'].values - test_data_df['Predicted_Reliability'].values))
queue_overflow_mae = np.mean(abs(test_data_df['Queue_Overflow_Prob'].values - test_data_df['Predicted_Queue_Overflow_Prob'].values))
incr_rcvd_mae = np.mean(abs(test_data_df['Incorrectly_Rcvd_Prob'].values - test_data_df['Predicted_Incr_Rcvd_Prob'].values))
delay_excd_mae = np.mean(abs(test_data_df['Delay_Excd_Prob'].values - test_data_df['Predicted_Delay_Excd_Prob'].values))
reliability_maxae = np.max(abs(test_data_df['Reliability'].values - test_data_df['Predicted_Reliability'].values))
queue_overflow_maxae = np.max(abs(test_data_df['Queue_Overflow_Prob'].values - test_data_df['Predicted_Queue_Overflow_Prob'].values))
incr_rcvd_maxae = np.max(abs(test_data_df['Incorrectly_Rcvd_Prob'].values - test_data_df['Predicted_Incr_Rcvd_Prob'].values))
delay_excd_maxae = np.max(abs(test_data_df['Delay_Excd_Prob'].values - test_data_df['Predicted_Delay_Excd_Prob'].values))

# Print results
print("Reliability - Accuracy: {}, MAE: {}, MaxAE: {}".format(reliability_accuracy, reliability_mae, reliability_maxae))
print("Failure Mode - Accuracy: {}".format(failure_mode_accuracy))
print("Queue Overflow - MeanAE: {}, MaxAE: {}".format(queue_overflow_mae, queue_overflow_maxae))
print("Incorrectly Received - MeanAE: {}, MaxAE: {}".format(incr_rcvd_mae, incr_rcvd_maxae))
print("Delay Exceeded - MeanAE: {}, MaxAE: {}".format(delay_excd_mae, delay_excd_maxae))

# Save results to file
# test_data_df.to_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_MultiModulation_Hovering_NoVideo/Multi_Modulation_Test_Cases_2_Downlink_LR_RESULTS.csv")

30/30 [==============================] - 0s 2ms/step
Reliability - Accuracy: 0.8802083333333334, MAE: 0.06897920702129251, MaxAE: 0.920998875623137
Failure Mode - Accuracy: 0.7395833333333334
Queue Overflow - MeanAE: 0.10290606482069699, MaxAE: 0.7845321993990133
Incorrectly Received - MeanAE: 0.010462100179784497, MaxAE: 0.06633887457962777
Delay Exceeded - MeanAE: 0.0204403356582099, MaxAE: 0.1983222501800813


In [15]:
np.mean([0.10290606482069699, 0.010462100179784497, 0.0204403356582099])

0.044602833552897125

## Test using Taguchi Test Dataset (No SINR)

In [5]:
# Set minimum probability for failure mode to be considered
MIN_FAILURE_PROB = 0.01

# Load test dataset
test_data_df = pd.read_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_MultiModulation_Hovering_Video/Test/Multi_Modulation_Test_Cases_1_Video.csv")

h_dist = test_data_df["Horizontal_Distance"].values
height = test_data_df["Height"].values

# Normalize inputs
max_height = 300
min_height = 60
max_h_dist = 1200
min_h_dist = 0

norm_h_dist = [2*(h_d-min_h_dist) / (max_h_dist-min_h_dist) - 1 for h_d in h_dist]
norm_height = [2*(h-min_height) / (max_height-min_height) - 1 for h in height]
norm_uav_send_int = test_data_df["UAV_Sending_Interval"].replace({10:-1, 20:-0.5, 40:0, 100:0.5, 1000:1}).values
norm_modulation = test_data_df["Modulation"].replace({"BPSK":1, "QPSK":0.3333, "QAM16":-0.3333, "QAM64":-1}).values

# For storing prediction results
predicted_reliability = []
predicted_incr_rcvd = [] 
predicted_delay_excd = []
predicted_queue_overflow = []

# Run inference
model_inputs = list(zip(norm_h_dist, norm_height, norm_uav_send_int, norm_modulation))
predicted_reliability = reliability_model.predict(model_inputs)
predicted_incr_rcvd = incr_rcvd_model.predict(model_inputs)
predicted_delay_excd = delay_excd_model.predict(model_inputs)
predicted_queue_overflow = queue_overflow_model.predict(model_inputs)
# print(prediction)

# Save the results to CSV
test_data_df['Predicted_Reliability'] = predicted_reliability
test_data_df['Predicted_Queue_Overflow_Prob'] = predicted_queue_overflow
test_data_df['Predicted_Incr_Rcvd_Prob'] = predicted_incr_rcvd
test_data_df['Predicted_Delay_Excd_Prob'] = predicted_delay_excd

test_data_df["Reliability_Class"] = pd.cut(test_data_df["Reliability"], bins=[-0.1,0.2,0.5,0.8,1], labels=["Low", "ModeratelyLow", "ModeratelyHigh", "High"])
test_data_df["Predicted_Reliability_Class"] = pd.cut(test_data_df["Predicted_Reliability"], bins=[-0.1,0.2,0.5,0.8,1], labels=["Low", "ModeratelyLow", "ModeratelyHigh", "High"])
test_data_df["Failure_Mode"] = test_data_df[["Queue_Overflow_Prob", "Incorrectly_Rcvd_Prob", "Delay_Excd_Prob"]].idxmax(axis=1)
test_data_df["Predicted_Failure_Mode"] = test_data_df[["Predicted_Queue_Overflow_Prob", "Predicted_Incr_Rcvd_Prob", "Predicted_Delay_Excd_Prob"]].idxmax(axis=1)
# Replace label for Failure Mode with "None" if none of the failure modes have a probability > 5%
test_data_df.loc[(test_data_df["Queue_Overflow_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Incorrectly_Rcvd_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Delay_Excd_Prob"] < MIN_FAILURE_PROB),["Failure_Mode"]] = "None"
test_data_df.loc[(test_data_df["Predicted_Queue_Overflow_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Predicted_Incr_Rcvd_Prob"] < MIN_FAILURE_PROB) & (test_data_df["Predicted_Delay_Excd_Prob"] < MIN_FAILURE_PROB),["Predicted_Failure_Mode"]] = "None"

# Compute the model accuracy and mean abs err
failure_mode = test_data_df["Failure_Mode"].replace({"Queue_Overflow_Prob":1, "Incorrectly_Rcvd_Prob":2, "Delay_Excd_Prob":3, "None":4})
failure_mode_predicted = test_data_df["Predicted_Failure_Mode"].replace({"Predicted_Queue_Overflow_Prob":1, "Predicted_Incr_Rcvd_Prob":2, "Predicted_Delay_Excd_Prob":3, "None":4})
reliability_accuracy = accuracy_score(test_data_df["Reliability_Class"], test_data_df["Predicted_Reliability_Class"])
failure_mode_accuracy = accuracy_score(failure_mode, failure_mode_predicted)
reliability_mae = np.mean(abs(test_data_df['Reliability'].values - test_data_df['Predicted_Reliability'].values))
queue_overflow_mae = np.mean(abs(test_data_df['Queue_Overflow_Prob'].values - test_data_df['Predicted_Queue_Overflow_Prob'].values))
incr_rcvd_mae = np.mean(abs(test_data_df['Incorrectly_Rcvd_Prob'].values - test_data_df['Predicted_Incr_Rcvd_Prob'].values))
delay_excd_mae = np.mean(abs(test_data_df['Delay_Excd_Prob'].values - test_data_df['Predicted_Delay_Excd_Prob'].values))
reliability_maxae = np.max(abs(test_data_df['Reliability'].values - test_data_df['Predicted_Reliability'].values))
queue_overflow_maxae = np.max(abs(test_data_df['Queue_Overflow_Prob'].values - test_data_df['Predicted_Queue_Overflow_Prob'].values))
incr_rcvd_maxae = np.max(abs(test_data_df['Incorrectly_Rcvd_Prob'].values - test_data_df['Predicted_Incr_Rcvd_Prob'].values))
delay_excd_maxae = np.max(abs(test_data_df['Delay_Excd_Prob'].values - test_data_df['Predicted_Delay_Excd_Prob'].values))

# Print results
print("Reliability - Accuracy: {}, MAE: {}, MaxAE: {}".format(reliability_accuracy, reliability_mae, reliability_maxae))
print("Failure Mode - Accuracy: {}".format(failure_mode_accuracy))
print("Queue Overflow - MeanAE: {}, MaxAE: {}".format(queue_overflow_mae, queue_overflow_maxae))
print("Incorrectly Received - MeanAE: {}, MaxAE: {}".format(incr_rcvd_mae, incr_rcvd_maxae))
print("Delay Exceeded - MeanAE: {}, MaxAE: {}".format(delay_excd_mae, delay_excd_maxae))

# Save results to file
test_data_df.to_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_MultiModulation_Hovering_Video/Test/Multi_Modulation_Test_Cases_1_Video_LR_NoSINR_RESULTS.csv")

30/30 [==============================] - 0s 741us/step


30/30 [==============================] - 0s 700us/step
Reliability - Accuracy: 0.8333333333333334, MAE: 0.08953863330088005, MaxAE: 0.9379441402414265
Failure Mode - Accuracy: 0.8125
Queue Overflow - MeanAE: 0.13154522967914598, MaxAE: 0.7391756660798018
Incorrectly Received - MeanAE: 0.0156993025721583, MaxAE: 0.07332729279695326
Delay Exceeded - MeanAE: 0.026987886067633433, MaxAE: 0.38342976538114404
